# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer

In [2]:
spark = SparkSession.builder.appName('crew').getOrCreate()

In [ ]:
#Example on how to use StringIndexer
df = spark.createDataFrame(
    [(0, "a"), (1, "b"), (2, "c"), (3, "a"), (4, "a"), (5, "c")],
    ["id", "category"])

indexer = StringIndexer(inputCol="category", outputCol="categoryIndex")
indexed = indexer.fit(df).transform(df)
indexed.show()

In [3]:
data = spark.read.csv('cruise_ship_info.csv', inferSchema=True, header=True)

In [4]:
data.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [5]:
data.groupBy('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [6]:
indexer = StringIndexer(inputCol='Cruise_line', outputCol='Cruise_indexer')
indexed = indexer.fit(data).transform(data)
indexed.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+--------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_indexer|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+--------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|          16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|          16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|           1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|           1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|           1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|        

In [ ]:
# What information should I use as input
#Tonnage, Passengers, Length, Cabins, Passenger_density, cruise_indexer
# My target
#crew

In [7]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [8]:
#Create an assembles
assembler = VectorAssembler(inputCols=['Age', 'Tonnage', 'passengers', 'length', 'passenger_density', 'Cruise_indexer'],
                            outputCol = 'features')

In [9]:
# Transform the data
output = assembler.transform(indexed)

In [10]:
output.head(1)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_indexer=16.0, features=DenseVector([6.0, 30.277, 6.94, 5.94, 42.64, 16.0]))]

In [11]:
# Select my final data
final_data = output.select(['features','crew'])

In [12]:
final_data.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [13]:
# Then we split our data
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [14]:
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               123|
|   mean|7.8244715447154585|
| stddev|3.6328761137410437|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [15]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                35|
|   mean| 7.687714285714284|
| stddev|3.0507656567551984|
|    min|              0.88|
|    max|             12.38|
+-------+------------------+



In [16]:
from pyspark.ml.regression import LinearRegression

In [17]:
# Create our linear regression model
lr = LinearRegression(labelCol='crew')

In [18]:
lr_model = lr.fit(train_data)

### See how well our model perform

In [19]:
test_results = lr_model.evaluate(test_data)

In [20]:
# The residuals is just a difference between the predictive value and the actual value from the test data
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| -0.5855946199997959|
| -0.5855946199997959|
|  0.8239187314241807|
| -1.2894344602148884|
|  1.3766750077927483|
|-0.02862972675692...|
|-0.16150529285988036|
| -1.3859495729646394|
|  0.3902711612677088|
|  1.3342487226849897|
| -1.4724646857143906|
| -0.1318366248747993|
|  0.5620352976727983|
| -0.8686016489788368|
|0.020962499330679396|
|  -1.552771379594346|
| -0.4063443929006709|
|   -0.34938586016691|
|0.047932273831176264|
|  0.9820113203760101|
+--------------------+
only showing top 20 rows



### Get some of those regression evalutaion metrics

In [21]:
# Root Mean Squared Error
test_results.rootMeanSquaredError

0.808612999369634

In [22]:
# r2
test_results.r2

0.9276809242068296

In [23]:
final_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              158|
|   mean|7.794177215189873|
| stddev|3.503486564627034|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [24]:
test_results.meanSquaredError

0.6538549827495558

In [25]:
test_results.meanAbsoluteError

0.6664964457799288

Because we got really good results, we need to double check that we really have a good results model.

We can check if the number of crew highly correlated with the number of passenger on board or as a number of crew hightly correlated with the number of cabins on board. If those things are true it makes sense that our model performed really well because there is two columns that are really highly correlated with the things I am trying to predict.

In [26]:
from pyspark.sql.functions import corr

In [27]:
data.describe().show()

+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|summary|Ship_name|Cruise_line|               Age|           Tonnage|       passengers|           length|            cabins|passenger_density|             crew|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|  count|      158|        158|               158|               158|              158|              158|               158|              158|              158|
|   mean| Infinity|       null|15.689873417721518| 71.28467088607599|18.45740506329114|8.130632911392404| 8.830000000000005|39.90094936708861|7.794177215189873|
| stddev|      NaN|       null| 7.615691058751413|37.229540025907866|9.677094775143416|1.793473548054825|4.4714172221480615| 8.63921711391542|3.503486564627034|
|    min|Adventure|    Azamara|   

We want to check if in my original data if the crew column is really related to the things, like the cabins.

In [28]:
data.select(corr('crew','passengers')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+



Getting a .91 value means that these two columns are highly correlated.

Then we will check againts cabins:

In [29]:
data.select(corr('crew','cabins')).show()

+------------------+
|corr(crew, cabins)|
+------------------+
|0.9508226063578497|
+------------------+



This 0.95 means those 2 columns are highly correlated.